In [3]:
cd home/karanverma

/home/karanverma


In [1]:
print('Starting of MLP training')
import logging
import datetime
import time
import os

import time
start_time = time.time()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# In[10]:

print(os.getcwd())
if not os.path.exists('/home/karanverma/approach_logs/MLP_classifier/'):
    os.makedirs('/home/karanverma/approach_logs/MLP_classifier/')

Starting of MLP training
/


In [2]:
import time
start_time = time.time()

In [3]:
import datetime
import numpy as np
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql import DataFrameStatFunctions as stat
import numpy as np
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import Window

import requests
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm
from pyspark.sql import DataFrameStatFunctions as stat
from pyspark.sql import Window
import pyspark.sql.functions as F
import os
import time
import json
import pickle

NIS_DATA_BASE_PATH = "gs://nis-segment-datasource-v3/processed/"
NIS_OLD_DATA_BASE_PATH = "gs://nis-localytics-datasource/processed/"

conf = SparkConf().setAll([('spark.driver.memory', '100g'), ('spark.broadcast.blockSize', '50m'), ("spark.executor.instances", '30')])
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 14:00:51 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/19 14:00:51 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/19 14:00:51 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/06/19 14:00:51 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [4]:
def get_raw_path(date, hours=None):
    paths = []
    base_path = NIS_RAW_DATA_BASE_PATH + date
    if not hours:
        return base_path + "/*/*.gz"
    for hour in hours:
        paths.append(base_path + "/" + str(hour).zfill(2) + "/*.gz")
    return ",".join(paths)

def process_raw_data(paths):
    def view_data_filters(x):
        x = x['properties']
        deviceid_filter = ('deviceId' in x) and (x['deviceId'] != '')
        time_filter = ('timeSpent' in x) and (int(x['timeSpent']) <= 100) and (int(x['timeSpent']) >= 0)
        return deviceid_filter and time_filter

    try:
        rdd = sc.textFile(paths) \
            .map(json.loads) \
            .filter(lambda x: "batch" in x).flatMap(lambda x: x["batch"]) \
            .filter(lambda x: ("event" in x) and (x["event"].lower() == "timespent-front")) \
            .filter(view_data_filters) \
            .map(lambda x: x['properties'])
        view_data = rdd.map(lambda x: (x['deviceId'], x['hashId'][:-2], x['timeSpent'])) \
            .toDF(['deviceId', 'hashId', 'timeSpent'])
        
        view_data = view_data.filter(view_data.timeSpent.isNotNull())
        #view_data = view_data.filter((getHashBucketUDF(view_data.deviceId) >= 16) & (getHashBucketUDF(view_data.deviceId) <= 25))
        view_data = view_data.groupby(view_data.deviceId, view_data.hashId) \
                             .agg(F.max(view_data.timeSpent).alias('overallTimeSpent'))
        
        return view_data
    except Exception as e:
        logger.warning("Error processing data: " + str(e))


In [5]:
from Utils import *
def getNewsData(d1, d2):
    newsMap = getNewsInDates(d1, d2)
    hashIdList = list(newsMap.keys())

    hashIdsWithFilter = []
    for h in hashIdList:
        if 'newsLanguage' in newsMap[h] and newsMap[h]['newsLanguage'] == 'english' and newsMap[h]['publishGroupList'][0]['countryCode'] == 'IN':
            hashIdsWithFilter.append(h.split('-')[0])
    
    return hashIdsWithFilter, newsMap


In [6]:
import logging

logger = logging.getLogger(str(datetime.datetime.today().date()))
hdlr = logging.FileHandler(
    '/home/karanverma/approach_logs/MLP_classifier/' + str(datetime.datetime.today().date()) + '.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.DEBUG)

In [7]:
def Log(s, flag=True):
    if flag:
        logger.info(s)
        print(s)

Log('----', flag=True)


----


In [6]:
sampled_dates = [datetime.datetime(2023,2,1,0,0,0), datetime.datetime(2023,2,6,0,0,0),datetime.datetime(2023,2,9,0,0,0),datetime.datetime(2023,2,14,0,0,0),
                datetime.datetime(2023,2,18,0,0,0),datetime.datetime(2023,2,20,0,0,0), datetime.datetime(2023,2,25,0,0,0)]
sampled_dates

[datetime.datetime(2023, 2, 1, 0, 0),
 datetime.datetime(2023, 2, 6, 0, 0),
 datetime.datetime(2023, 2, 9, 0, 0),
 datetime.datetime(2023, 2, 14, 0, 0),
 datetime.datetime(2023, 2, 18, 0, 0),
 datetime.datetime(2023, 2, 20, 0, 0),
 datetime.datetime(2023, 2, 25, 0, 0)]

In [7]:
train_days = 10
test_days = 7
embed_days = 30
n_days = train_days + test_days + embed_days
k = 0
st_date = (sampled_dates[k]+ datetime.timedelta(days=test_days)) - datetime.timedelta(days=n_days)


date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
dates_str_embed = dates_str[0:embed_days]
dates_str_train = dates_str[embed_days:embed_days+train_days+1]
dates_str_test = dates_str[-test_days:]
print("embed data dates : ", dates_str_embed, len(dates_str_embed))
print("train data dates : ",dates_str_train, len(dates_str_train) )
print("test data dates : ",dates_str_test, len(dates_str_test))
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000
print(dates[0], dates[-1])
hashIdsWithFilter, newsMap = getNewsData(dates[0], dates[-1])

embed data dates :  ['2022/12/23', '2022/12/24', '2022/12/25', '2022/12/26', '2022/12/27', '2022/12/28', '2022/12/29', '2022/12/30', '2022/12/31', '2023/01/01', '2023/01/02', '2023/01/03', '2023/01/04', '2023/01/05', '2023/01/06', '2023/01/07', '2023/01/08', '2023/01/09', '2023/01/10', '2023/01/11', '2023/01/12', '2023/01/13', '2023/01/14', '2023/01/15', '2023/01/16', '2023/01/17', '2023/01/18', '2023/01/19', '2023/01/20', '2023/01/21'] 30
train data dates :  ['2023/01/22', '2023/01/23', '2023/01/24', '2023/01/25', '2023/01/26', '2023/01/27', '2023/01/28', '2023/01/29', '2023/01/30', '2023/01/31', '2023/02/01'] 11
test data dates :  ['2023/02/02', '2023/02/03', '2023/02/04', '2023/02/05', '2023/02/06', '2023/02/07', '2023/02/08'] 7
2022-12-23 00:00:00 2023-02-08 00:00:00


In [16]:
#need this for live data else skip if backtesting
NIS_RAW_DATA_BASE_PATH = "gs://inshorts-segment-raw/data/segment-raw-v5a/"

path = get_raw_path(datetime.datetime.now().strftime("%Y/%m/%d"))
today_data = process_raw_data(path)
today_data = today_data.filter(today_data.hashId.isin(hashIdsWithFilter))

In [17]:
#generate datasets
def generate_data(datestr):
#     datestr = [d.strftime(date_fmt) for d in dates]
    paths = get_path(datestr, 'otherEvents')
#     print(sorted(paths), len(paths))
    data = sqlContext.read.parquet(*paths)

    data = filter_app(data, app_name=None)
    data = filter_tenant(data, tenant='en')

    # data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))

    data = data.select(data.deviceId, (F.split(data.hashId, '-')[0]).alias('hashId'), data.overallTimeSpent, data.notificationOpened, data.notificationShown, data.notificationType,data.eventName)
    data = data.filter( data.eventName.isin(['Notification Shown','Notification Opened']) )

    data = data.filter(data.hashId.isin(hashIdsWithFilter))
    
    data = data.select('deviceId', 'hashId' ,'overallTimeSpent', 'notificationOpened', 'notificationShown')\
            .groupby('deviceId', 'hashId')\
            .agg(F.max('overallTimeSpent').alias('timeSpent'), 
                 F.sum('notificationShown').alias('notiShown'),F.sum('notificationOpened').alias('notiOpened'))
    data = data.withColumn("notiShown",F.when(data.notiShown < data.notiOpened, 
                                                     data.notiOpened).otherwise(data.notiShown))
    return data

data_embed = generate_data(dates_str_embed)
data_train = generate_data(dates_str_train)
data_test = generate_data(dates_str_test)

NameError: name 'get_path' is not defined

In [ ]:
#to be used when live data is needed
data = data.union(today_data)
data.cache()

In [18]:
def popularity(data, newsMeanMap):
    Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap_train = defaultdict(lambda: (7, 1))
popularity(data, newsMe1banMap)

Computing news mean


In [ ]:
data_embed.printSchema()

In [46]:
data_comb = data_embed.union(data_train)
data_comb.groupBy('eventName').count().orderBy('count').show()

23/06/11 01:47:04 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1115.7 KiB


+-------------------+----------+
|          eventName|     count|
+-------------------+----------+
|Notification Opened| 176801120|
| Notification Shown|3024511245|
+-------------------+----------+



In [47]:
data_comb.filter(data_comb.notificationOpened > 0).groupBy('eventName').count().orderBy('count').show()

23/06/11 01:50:33 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1117.8 KiB


+-------------------+---------+
|          eventName|    count|
+-------------------+---------+
|Notification Opened|176801120|
+-------------------+---------+



In [ ]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA
import time

training_start = time.time()


# data_filtered = data.filter(F.udf(lambda hashId, noti: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))
# abcset=sqlContext.read.parquet("gs://pvtrough_asia_south1/tf_idf/devices_fulfilling_criteria")
# df_temp=abcset.filter(abcset.set_typ == 'roberta').select(['deviceid']).distinct()
# df_temp=df_temp.withColumnRenamed("deviceid","deviceId")

# data_filtered=data_filtered.join(df_temp,["deviceId"],"inner")

data_embed_filtered = data_embed.filter(data_embed.notiOpened>0)    


# Log("Training took %s minutes"%(int((time.time() - training_start) / 60)))

In [ ]:
d1 = data.select('deviceId').distinct().collect()
d2 = data_filtered.select('deviceId').distinct().collect()
d3 = df_temp.select('deviceId').distinct().collect()

In [56]:
len(d2)

367590

In [ ]:
from tqdm import tqdm
df_left = pd.DataFrame(columns = ['deviceId', 'cluster_left_out_date'])
left_out = set(d3) - set(d2)
pbar = tqdm(total=len(left_out))
cluster_left_out = (datetime.datetime.now() + datetime.timedelta(days=0)) .strftime("%Y_%m_%d") 
i = 0
for d in left_out:
    df_left.loc[i,'deviceId'] = d['deviceId']
    df_left.loc[i,'cluster_left_out_date'] = cluster_left_out
    i= i+1
    pbar.update(1)
pbar.close()

In [23]:
dfl = df_left.copy()
print(dfl.head())

                               deviceId cluster_left_out_date
0  c1482731-233c-4cab-a4b0-7cc7dbdaeb94            2023_06_09
1  7ed92573-0d52-4434-9ee9-8c457614942e            2023_06_09
2  da0fecf2-f57f-4b16-86ad-8c9c10255d05            2023_06_09
3  2018ac32-1dce-4d7a-8de1-42ea3f42103e            2023_06_09
4  135af132-3d63-4cc3-bccf-f5dc3ec15e30            2023_06_09


In [ ]:

path_leftout = 'gs://pvtrough_asia_south1/clustering/leftout_devices_roberta'
df_left = sqlContext.createDataFrame(dfl[['deviceId', 'cluster_left_out_date']])
df_left.coalesce(1).write.partitionBy('cluster_left_out_date').csv(path_leftout , sep=',', header=True,mode = 'append')

    

In [ ]:
import torch.nn as nn
from torch.nn.parallel import DataParallel
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback


model_name = "roberta-base"
num_classes = 17
# model_path = "output/checkpoint-12000"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes,output_hidden_states=True)

# Move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Check the number of available GPUs
if torch.cuda.device_count() > 1:
    # Specify which GPUs to use
    device_ids = [0, 1]  # Adjust the GPU IDs based on your system configuration
    model = DataParallel(model, device_ids=device_ids)
    print(isinstance(model, DataParallel))


In [ ]:
def get_embedding(model, tokenizer, text):
    model.eval()

    with torch.no_grad():
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
        outputs = model(input_ids=input_ids)
        hidden_states = outputs.hidden_states
        last_layer_hidden_states = hidden_states[-1]
        embeddings = torch.mean(last_layer_hidden_states, dim=1).squeeze().cpu().numpy()

    return embeddings

In [12]:
from pyspark.sql.functions import array, col
path_embedding = 'gs://pvtrough_asia_south1/clustering/MLP_embedding'
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)
# df_embedding.take(1)

In [ ]:
newsMap['l_ebpzzatk-1']

In [ ]:
#generate news embeddings based on notification text
newsMapProcessed = {}
document_embeddings = {}
try:
    already_embedded = df_embedding.select('hid').distinct().collect()
    already_embedded = [x['hid'] for x in already_embedded]
except:
    print("News Embeddings not found, generating new one")
    already_embedded = []
for hId in tqdm(newsMap):
    
    h = hId.split('-')[0]
#     print(h)
    if h in already_embedded or (h not in hashIdsWithFilter) :
        continue
    else:    
        newsMapProcessed[h] = {}
        newsMapProcessed[h]['title'] = newsMap[hId]['title']
        newsMapProcessed[h]['content'] = newsMap[hId]['content']
        newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] #+ "." + newsMapProcessed[h]['content']
        document_embeddings[h] = get_embedding(model, tokenizer, newsMapProcessed[h]['features'])
     

In [ ]:
len(document_embeddings)

In [ ]:
if len(document_embeddings) > 0:
    df_em_pandas = pd.DataFrame(document_embeddings).T
    df_em_pandas['hid'] = df_em_pandas.index
    df_em_pandas['embed_date'] = sampled_dates[0]
    df_em_spark = (sqlContext.createDataFrame(df_em_pandas))
    df_em_spark.write.partitionBy('embed_date').csv(path_embedding , sep=',', header=True,mode = 'append')

In [13]:

df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)
df_embedding = df_embedding.distinct()
feat_cols = [str(x) for x in range(0,len(df_embedding.columns) - 2)]
df_embedding_exploded = df_embedding.distinct()
df_embedding = df_embedding.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))
df_embedding.take(1)

23/06/19 14:07:35 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(hid='ub2769mj', embedding=[0.056168098002672195, 0.09086097776889801, 0.02046060748398304, 0.03756670653820038, -0.07304298132658005, -0.1654847264289856, 0.08309705555438995, -0.06807274371385574, 0.057520072907209396, -0.10866157710552216, 0.05438607558608055, -0.0772598460316658, -0.1515483260154724, -0.14197292923927307, 0.09427487850189209, -0.07601520419120789, 0.20039351284503937, -0.00022923845972400159, 0.1356627196073532, 0.09170513600111008, -0.10971157252788544, -0.02387220598757267, -0.05697789415717125, -0.026600142940878868, 0.024021238088607788, -0.055152758955955505, -0.0244436077773571, 0.026310209184885025, -0.2144644558429718, -0.06618518382310867, -0.02446378953754902, 0.1300676316022873, 0.047172632068395615, -0.1647927612066269, 0.05774666741490364, 0.06204885616898537, 0.12978771328926086, 0.08066125959157944, 0.23324306309223175, -0.020405832678079605, -0.065200075507164, -0.1806843876838684, 0.12990179657936096, -0.0006229899008758366, -0.0738190859556198

In [ ]:
from pyspark.sql.functions import *

k = 0
data_filtered = data_embed_filtered
# # calculate idf_weight
N = data_filtered.select('hashId').distinct().count()
users = data_filtered.groupBy('hashId').agg(F.countDistinct('deviceId').alias('users'))
idf_weights = users.withColumn('idf_weight', F.log(N / F.col('users')))

# Multiply embeddings with IDF weights
df_result = df_embedding.join(idf_weights, df_embedding.hid == idf_weights.hashId)
df_result = df_result.withColumn('weighted_embedding', F.expr('transform(embedding, (x, i) -> x * idf_weight)'))


df_result = df_result.select('hid','weighted_embedding')
df_joined = data_filtered.join(df_result, data_filtered.hashId == df_result.hid , 'left')


# assuming your dataframe is called df
grouped_df = df_joined.groupBy("deviceId").agg(*[
    expr(f"avg(weighted_embedding[{i}]) as avg_{i}") for i in range(768)
])



# assuming your dataframe is called grouped_df
avg_cols = [col(f"avg_{i}") for i in range(768)]
grouped_df = grouped_df.withColumn("topic_avg_embedding", array(*avg_cols))

path_vec = "gs://pvtrough_asia_south1/clustering/MLP_device_vectors_"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
path_vec_exploded = "gs://pvtrough_asia_south1/clustering/MLP_device_vectors_exploded_"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
print(path_vec, path_vec_exploded)

# grouped_df.select('deviceId', 'topic_avg_embedding').write.parquet(path=path_vec, mode='overwrite')
grouped_df.select('deviceId', *avg_cols).write.parquet(path=path_vec_exploded, mode='overwrite')

In [9]:
from pyspark.sql.functions import array_contains, size, col
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

k = 0
num_clusters = 1000
path_vec = "gs://pvtrough_asia_south1/clustering/MLP_device_vectors_"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
# path_vec = "gs://pvtrough_asia_south1/clustering/device_vectors_roberta_live/"
print(path_vec)
# print(path_vec)

temp = sqlContext.read.parquet(path_vec)
df = temp.filter(~array_contains(temp.topic_avg_embedding, float('nan')))

# df = temp.sample(fraction=0.4)

df = df.dropna(how='all')
df = df.where(size(col("topic_avg_embedding")) == 768)



# to_vector_udf = udf(lambda arr: Vectors.dense(arr), VectorUDT())
# df_with_vector = df.select("deviceId", to_vector_udf("topic_avg_embedding").alias("features"))

# kmeans = KMeans().setK(num_clusters).setSeed(1).setFeaturesCol('features')
# model = kmeans.fit(df_with_vector)

# c = model.transform(df_with_vector)

# print(path_clu)
# dfdump = c.withColumn('cluster', col('prediction').cast("bigint"))
# dfdump.select('deviceId','cluster').write.parquet(path=path_clu, mode='overwrite')



23/06/19 14:06:21 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


gs://pvtrough_asia_south1/clustering/MLP_device_vectors_2023_02_01/


Exception in thread "serve RDD 2 with partitions 0" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [14]:
df_embedding.show()

+----------+--------------------+
|       hid|           embedding|
+----------+--------------------+
|  ub2769mj|[0.05616809800267...|
|  e8w8v3zb|[0.04845666512846...|
|  kzmreakp|[-0.0056108236312...|
|  va5zlabn|[-0.0380604974925...|
|  vnf9f7p4|[-0.0076963086612...|
|  us41p82b|[0.11811468005180...|
|  bqrk6wn4|[-0.0150790289044...|
|  es56qxx1|[0.02011904120445...|
|  mqvnamsz|[0.03333922475576...|
|  ypvuqx8s|[-0.0207866206765...|
|  iiyrnpnp|[0.00308009143918...|
|  4v2hvtbn|[0.01892139948904...|
|  ujqnhtpc|[0.02996562793850...|
|  np9wyxda|[0.01796540617942...|
|  kvnugdno|[-0.0077103255316...|
|l_aaf9b9z7|[0.10551590472459...|
|  ebe5fni7|[-0.0012556678848...|
|  t7hukgw9|[0.02591428719460...|
|l_fla6gji6|[0.05237537622451...|
|  jiic81uu|[0.06345029175281...|
+----------+--------------------+
only showing top 20 rows



In [40]:
data_train.select('hashId').take(3)

[Row(hashId='zuoolx0y'), Row(hashId='niglsrbe'), Row(hashId='76imqnpu')]

In [ ]:
train_data = data_train.join(df,["deviceId"],"inner")
train_data = train_data.join(df_embedding,train_data.hashId == df_embedding.hid,"inner")
train_data = train_data.select('deviceId', 'hashId', 'topic_avg_embedding', 'embedding','notiOpened')
train_data.printSchema()

In [ ]:
train_data = train_data.dropna(how='all')
train_data = train_data.where((size(col("topic_avg_embedding")) == 768) & (size(col("embedding")) == 768))
train_data = train_data.withColumn("notiOpened",F.when(train_data.notiOpened > 1, 
                                                     1).otherwise(train_data.notiOpened))
# train_data.count()

In [ ]:
from pyspark.sql.functions import col, concat

train_data_features = train_data.withColumn('embedding_features', 
                                            concat(col('topic_avg_embedding'), col('embedding')))\
                                .select('embedding_features', 'notiOpened')
                                    

In [ ]:
train_data_features.show()

In [ ]:
d1 = data_train.select('deviceId').distinct().collect()
d2 = train_data.select('deviceId').distinct().collect()

In [ ]:
left_out = set(d1) - set(d2)
len(left_out)

In [20]:
merged_df = train_data.sample(fraction = 0.01)

In [26]:
assembler = VectorAssembler(inputCols = ['embedding_features'], outputCol = 'vector_features')
train_data_vector = assembler.transform(train_data_features)

IllegalArgumentException: Data type array<double> of column embedding_features is not supported.

In [25]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import expr, udf
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


data = train_data_features.sample(fraction = 0.1).select("embedding_features", "notiOpened")

# Split the data into training and testing sets
(training_data, testing_data) = data.randomSplit([0.7, 0.3])

# Define the deep learning model architecture
layers = [768, 768, 2]  # Adjust the layer sizes as needed
classifier = MultilayerPerceptronClassifier(layers=layers, labelCol="notiOpened", featuresCol="embedding_features")

# Train the model
model = classifier.fit(training_data)

# # Make predictions on the testing data
# predictions = model.transform(testing_data)

# # Evaluate the model
# evaluator = BinaryClassificationEvaluator(labelCol="notiOpened", rawPredictionCol="prediction")
# areaUnderROC = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
# print("Area under ROC curve:", areaUnderROC)


IllegalArgumentException: requirement failed: Column embedding_features must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.ArrayType:array<double>.

In [32]:
from pymongo import UpdateOne
import datetime
import logging
import os
import shutil
import sys
import json

import numpy as np
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *
from Utils import *
from tqdm import tqdm

mongoClient = getMongoClient()
trainDB = mongoClient['trialDB_roberta']
deviceVectorCollection = trainDB['deviceVectors']
deviceClusterCollection = trainDB['notiDeviceClusters']

newsVectorCollection = trainDB['newsVectors']
newsTSpentCollection = trainDB['newsTSpent']
newsScoreCollection = trainDB['newsScores']
deviceNotiClustersCollection = trainDB['notiDeviceClusters']

clusterCentersCollection = trainDB['clusterCenters']
 
def getDeviceVectorsFromMongo(deviceIds):
    
    deviceVectorMap = {}
    cursor = deviceVectorCollection.find({"_id" : { "$in" : deviceIds }})
    
    for c in cursor:
        deviceVectorMap[c['_id']] = np.array(c['vector'])
    
    return deviceVectorMap
        
def getNewsVectorsFromMongo(hashIds):
    newsVectorMap = {}
    cursor = newsVectorCollection.find({"_id" : { "$in" : hashIds }})
    
    for c in cursor:
        newsVectorMap[c['_id']] = np.array(c['vector'])
    
    return newsVectorMap


def insertDeviceVectorsInMongo(deviceVectors):
    
    collection = deviceVectorCollection
    ops = []
    results = []
    for deviceId in tqdm(deviceVectors):
        key = {"_id" : deviceId}
        devData = {"$set" : {"vector" : deviceVectors[deviceId]}}
        ops.append(UpdateOne(key, devData, upsert=True))
    
        if len(ops) == 100000:
            results.append(collection.bulk_write(ops,ordered=False))
            ops = []

    if len(ops) > 0:
        results.append(collection.bulk_write(ops,ordered=False))

def insertNewsVectorsInMongo(newsVectors):
    for hashId in newsVectors:
        key = {"_id" : hashId}
        newsData = {"$set" : {"vector" : list(newsVectors[hashId])}}
        newsVectorCollection.update_one(key, newsData, upsert=True)
        
        
oldClusters = {}
def getDeviceClustersFromMongo(deviceIds):
    print("here: ", deviceClusterCollection)
    deviceClusterMap = {}
    if len(deviceIds) == 0:
        cursor = deviceClusterCollection.find()
    else:
        cursor = deviceClusterCollection.find({"_id" : { "$in" : deviceIds }})
    
    for c in cursor:
        deviceClusterMap[c['_id']] = c['cluster']
        oldClusters[c['_id']] = c['cluster']
    
    return deviceClusterMap

"""deprecated"""
# def insertDeviceClustersInMongo(deviceClusters):
#     collection = deviceClusterCollection
#     ops = []
#     results = []
    
#     for deviceId in deviceClusters:
#         key = {"_id" : deviceId}
        
#         devData = {"$set" : {"cluster" : int(deviceClusters[deviceId])}}
        
#         if deviceId not in oldClusters or int(deviceClusters[deviceId]) != oldClusters[deviceId]:
#             devData = {"$set" : {"cluster" : int(deviceClusters[deviceId]), "updatedAt" : datetime.datetime.now()}}
            
#         ops.append(UpdateOne(key, devData, upsert=True))
    
#         if len(ops) == 1000:
#             results.append(collection.bulk_write(ops,ordered=False))
#             ops = []

#     if len(ops) > 0:
#         results.append(collection.bulk_write(ops,ordered=False))
        


def insertNewsVectorsInMongo(newsVectors):
    for hashId in newsVectors:
        key = {"_id" : hashId}
        newsData = {"$set" : {"vector" : list(newsVectors[hashId])}}
        newsVectorCollection.update_one(key, newsData, upsert=True)
        
def getNewsTSpentFromMongo(hashIds):
    newsTSpentMap = {}
    cursor = newsTSpentCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['tSpent']:
            data[int(cluster)] = c['tSpent'][cluster]
        newsTSpentMap[c['_id']] = data
        
    return newsTSpentMap

def insertNewsTSpentInMongo(newsTSpentMap):
    for hashId in newsTSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsTSpentMap[hashId]:
            data[str(cluster)] = newsTSpentMap[hashId][cluster]
        tSpentData = {"$set" : {"tSpent" : data}}
        newsTSpentCollection.update_one(key, tSpentData, upsert=True)

def getNewsScoreInMongo(hashIds):
    newsScoreMap = {}
    cursor = newsScoreCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['tSpent']:
            data[int(cluster)] = c['tSpent'][cluster]
        newsScoreMap[c['_id']] = data
        
    return newsScoreMap

def insertNewsScoreInMongo(newsTSpentMap):
    for hashId in newsTSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsTSpentMap[hashId]:
            if newsTSpentMap[hashId][cluster][1] > 1:
                data[str(cluster)] = newsTSpentMap[hashId][cluster][0] / newsTSpentMap[hashId][cluster][1]  
        
        scoreData = {"$set" : {"tSpent" : data, "updatedAt" : datetime.datetime.now()}}
        newsScoreCollection.update_one(key, scoreData, upsert=True)

def insertDeviceClustersInMongo(deviceClusters, oldClusters, Log):
    ops = []
    #results = []
    
    cntUpdated = 0
    for deviceId in deviceClusters:
        key = {"_id" : deviceId}
        
        devData = {"$set" : {"cluster" : int(deviceClusters[deviceId]), "updatedAt" : datetime.datetime.now()}}
        
        if deviceId in oldClusters and deviceClusters[deviceId] == oldClusters[deviceId]:
            continue
        
        cntUpdated += 1
        ops.append(UpdateOne(key, devData, upsert=True))
    
        if len(ops) == 100000:
            try:
                result = deviceNotiClustersCollection.bulk_write(ops,ordered=False)
                ops = []
            except Exception as e:
                ops = []
                Log("Failed to insert devices clusters, " + str(e))
        
        if cntUpdated % 100000 == 0:
            Log("Inserted " + str(cntUpdated) + " Device Clusters")
                
    if len(ops) > 0:
        deviceNotiClustersCollection.bulk_write(ops,ordered=False)
    
    print(cntUpdated,len(deviceClusters))
    Log("Updated  device clusters")


In [33]:

# from ColabUtils import *
device_vect_df = df.toPandas()
device_vect_df

deviceVectors = {}
for v in tqdm(device_vect_df.values):
    deviceVectors[v[0]] = v[1]
insertDeviceVectorsInMongo(deviceVectors)

100%|██████████| 367590/367590 [04:26<00:00, 1380.60it/s] 


In [34]:
labels = dfdump.select('deviceId','cluster').toPandas()
labels = pd.Series(labels.cluster.values,index=labels.deviceId).to_dict()
d4 = labels.keys()

23/06/09 11:47:50 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB


In [35]:

newClusterWiseDevices = defaultdict(lambda: set())
for deviceId, cluster in tqdm(labels.items()):
    newClusterWiseDevices[cluster].add(deviceId)

100%|██████████| 367590/367590 [00:00<00:00, 2298282.17it/s]


In [36]:
# from ColabUtils import *
deviceClusterMapExisiting = getDeviceClustersFromMongo([])

here:  Collection(Database(MongoClient(host=['171.16.11.97:27017', '171.16.11.96:27017', '171.16.11.94:27017'], document_class=dict, tz_aware=False, connect=True), 'trialDB_roberta'), 'notiDeviceClusters')


In [37]:
clusterWiseDevices = defaultdict(lambda: set())
for d in tqdm(deviceClusterMapExisiting):
    clusterWiseDevices[deviceClusterMapExisiting[d]].add(d)

100%|██████████| 414697/414697 [00:00<00:00, 1830134.46it/s]


In [38]:

marked = {}
clusterMap = {}
for x in newClusterWiseDevices:
    mxinter = -1
    mxidx = -1
    for y in clusterWiseDevices:
        if y in marked:
            continue
        inter = (len(clusterWiseDevices[y].intersection(newClusterWiseDevices[x])) / len(clusterWiseDevices[y])*100)
        if inter > mxinter:
            mxinter = inter
            mxidx = y
            
    if mxidx < 0:
        mxidx = x
    
    marked[mxidx] = 1
    clusterMap[x] = mxidx
    Log("Mapping cluster " + str(x) + " to " + str(mxidx) + ", %age intersection = " + str(mxinter))

Mapping cluster 497 to 837, %age intersection = 23.380281690140844
Mapping cluster 952 to 673, %age intersection = 42.70833333333333
Mapping cluster 722 to 367, %age intersection = 20.567986230636834
Mapping cluster 663 to 607, %age intersection = 18.21735731039875
Mapping cluster 671 to 233, %age intersection = 20.98765432098765
Mapping cluster 159 to 987, %age intersection = 26.141450313339305
Mapping cluster 459 to 959, %age intersection = 27.842003853564545
Mapping cluster 446 to 461, %age intersection = 13.358778625954198
Mapping cluster 94 to 626, %age intersection = 15.86737714624038
Mapping cluster 611 to 580, %age intersection = 30.317460317460316
Mapping cluster 386 to 991, %age intersection = 12.392638036809817
Mapping cluster 160 to 77, %age intersection = 10.51912568306011
Mapping cluster 698 to 532, %age intersection = 19.220945083014048
Mapping cluster 602 to 733, %age intersection = 18.579234972677597
Mapping cluster 58 to 386, %age intersection = 23.603192702394526
Map

In [39]:
updatedDeviceClusters = {}
for cluster in newClusterWiseDevices:
    for deviceId in tqdm(newClusterWiseDevices[cluster]):
        updatedDeviceClusters[deviceId] = clusterMap[cluster]

100%|██████████| 1/1 [00:00<00:00, 29959.31it/s]


In [41]:
insert_start = time.time()
# from ColabUtils import *
insertDeviceClustersInMongo(updatedDeviceClusters, deviceClusterMapExisiting, Log)

Inserted 100000 Device Clusters
Inserted 200000 Device Clusters
290297 367590
Updated  device clusters


In [145]:
insert_time = (time.time() - insert_start)

Log("Insertion took %s percent of time" % (100 * insert_time / (time.time() - start_time)))

Insertion took 0.2862641270220547 percent of time


In [42]:
dataWithClusters = data_filtered.select('hashId', 'overallTimeSpent', F.udf(lambda deviceId: updatedDeviceClusters[deviceId], IntegerType())('deviceId').alias('cluster'))
newsClusterMean = dataWithClusters.groupby('hashId', 'cluster').agg(F.sum('overallTimeSpent'),F.count('hashId'))
newsClusterMeanDf = newsClusterMean.toPandas()

23/06/09 11:50:14 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1005.8 KiB


In [43]:
newsTSpentMap = defaultdict(lambda: defaultdict(lambda: (7, 1)))
for v in tqdm(newsClusterMeanDf.values):
    newsTSpentMap[v[0]][v[1]] = (v[2], v[3])

100%|██████████| 3866744/3866744 [00:04<00:00, 902416.81it/s]


In [44]:
Log("Inserting News Score Data")
insert_start = time.time()
insertNewsTSpentInMongo(newsTSpentMap)
insertNewsScoreInMongo(newsTSpentMap)
insert_time = (time.time() - insert_start)
Log("Inserting News Score took %s percent of time" % str(100 * insert_time / (time.time() - start_time)))

Inserting News Score Data
Inserting News Score took 0.8065278979314444 percent of time


In [45]:
def get_raw_path_notification(date, hours=None):
    paths = []
    base_path = NIS_RAW_DATA_BASE_PATH + date
    if not hours:
        return base_path + "/*/*.gz"
    for hour in hours:
        paths.append(base_path + "/" + str(hour).zfill(2) + "/*.gz")
    return ",".join(paths)

def process_raw_data_notification(paths):
    def view_data_filters(x):
        x = x['properties']
        deviceid_filter = ('device_id' in x) and (x['device_id'] != '')
        noti_filter = ('notiOpened' in x) and (int(x['timeSpent']) <= 100)
        return deviceid_filter and time_filter

    try:
        rdd = sc.textFile(paths)             .map(json.loads)   .filter(lambda x: ("event_name" in x) and ( (x["event_name"].lower() == "notification opened") or (x["event_name"].lower() == "notification shown") ))            
        view_data = rdd.map(lambda x: (x['device_id'], x['hashId'][:-2], x['short_time'], x['noti_opened'], x['noti_shown'], x['event_name']))             .toDF(['deviceId', 'hashId', 'timeSpent', 'notiOpened', 'notiShown', 'event_name'])
        
        view_data = view_data.withColumn("notiOpened",F.when(view_data.event_name == 'Notification Opened', 1).otherwise(0))
        view_data = view_data.withColumn("notiShown",F.when(view_data.event_name == 'Notification Shown', 1).otherwise(0))
        view_data = view_data.select('deviceId', 'hashId', 'timeSpent', 'notiOpened', 'notiShown')  .groupby(view_data.deviceId, view_data.hashId)                              .agg(F.max(view_data.timeSpent).alias('timeSpent'),F.sum(view_data.notiOpened).alias('notiOpened'),F.sum(view_data.notiShown).alias('notiShown'))
        view_data = view_data.withColumn("notiShown",F.when(view_data.notiShown < view_data.notiOpened, 
                                                     view_data.notiOpened).otherwise(view_data.notiShown))
        return view_data
    except Exception as e:
        logger.warning("Error processing data: " + str(e))


In [46]:
NIS_RAW_DATA_BASE_PATH = "gs://inshorts-minimal-event/data/inshorts-minimal-event-v1/"

path = get_raw_path_notification(datetime.datetime.now().strftime("%Y/%m/%d"))
today_data = process_raw_data_notification(path)
today_data = today_data.filter(today_data.hashId.isin(hashIdsWithFilter))

In [47]:
n_days = 7
st_date = datetime.datetime.now() - datetime.timedelta(days=n_days+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]

# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], datetime.datetime.now())

In [48]:
datestr = [d.strftime(date_fmt) for d in dates]
paths = get_path(datestr, 'otherEvents')

data = sqlContext.read.parquet(*paths)

data = filter_app(data, app_name=None)
data = filter_tenant(data, tenant='en')

# data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))

data = data.select(data.deviceId, (F.split(data.hashId, '-')[0]).alias('hashId'), data.overallTimeSpent, data.notificationOpened, data.notificationShown, data.notificationType,data.eventName)
data = data.filter( data.eventName.isin(['Notification Shown','Notification Opened']) )

data = data.filter(data.hashId.isin(hashIdsWithFilter))

In [49]:
data = data.select('deviceId', 'hashId' ,'overallTimeSpent', 'notificationOpened', 'notificationShown')\
            .groupby('deviceId', 'hashId')\
            .agg(F.max('overallTimeSpent').alias('timeSpent'), 
                 F.sum('notificationShown').alias('notiShown'),F.sum('notificationOpened').alias('notiOpened'))
data = data.withColumn("notiShown",F.when(data.notiShown < data.notiOpened, 
                                                     data.notiOpened).otherwise(data.notiShown))

In [50]:
data = data.union(today_data)
data.cache()

DataFrame[deviceId: string, hashId: string, timeSpent: double, notiShown: bigint, notiOpened: bigint]

In [61]:

df_temp1=data_filtered.select(['deviceid']).distinct()
df_temp1=df_temp.withColumnRenamed("deviceid","deviceId")

data=data.join(df_temp1,["deviceId"],"inner")

In [63]:
dataWithClusters = data.select('hashId', 'notiOpened','notiShown', F.udf(lambda deviceId: updatedDeviceClusters[deviceId], IntegerType())('deviceId').alias('cluster'))
newsClusterNotiMean = dataWithClusters.groupby('hashId', 'cluster').agg(F.sum('notiOpened'),F.sum('notiShown'),F.count('hashId'))
# newsClusterNotiMeanDf = newsClusterNotiMean.toPandas()

In [64]:
newsClusterNotiMean.show()

----------------------------------------                       (43 + 357) / 400]
Exception happened during processing of request from ('127.0.0.1', 50834)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/sock

Py4JError: An error occurred while calling o70252.showString

In [ ]:
newsNotiSpentCollection = trainDB['newsNotiSpent']
newsNotiScoreCollection = trainDB['newsNotiScores']


def getNewsNotiSpentFromMongo(hashIds):
    newsNotiSpentMap = {}
    cursor = newsNotiSpentCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['nOpened']:
            data[int(cluster)] = c['nOpened'][cluster]
        newsNotiSpentMap[c['_id']] = data
        
    return newsNotiSpentMap

def insertNewsNotiSpentInMongo(newsNotiSpentMap):
    for hashId in newsNotiSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsNotiSpentMap[hashId]:
            data[str(cluster)] = newsNotiSpentMap[hashId][cluster]
        NotiSpentData = {"$set" : {"nOpened" : data}}
        newsNotiSpentCollection.update_one(key, NotiSpentData, upsert=True)

def getNewsNotiScoreInMongo(hashIds):
    newsNotiScoreMap = {}
    cursor = newsNotiScoreCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['nOpened']:
            data[int(cluster)] = c['nOpened'][cluster]
        newsNotiScoreMap[c['_id']] = data
        
    return newsNotiScoreMap

def insertNewsNotiScoreInMongo(newsNotiSpentMap):
    for hashId in newsNotiSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsNotiSpentMap[hashId]:
            if newsNotiSpentMap[hashId][cluster][1] > 1:
                data[str(cluster)] = newsNotiSpentMap[hashId][cluster][0] / newsNotiSpentMap[hashId][cluster][1]  
        
        notiScoreData = {"$set" : {"nOpened" : data, "updatedAt" : datetime.datetime.now()}}
        newsNotiScoreCollection.update_one(key, notiScoreData, upsert=True)
        
        

23/06/09 11:53:57 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 27.0 in stage 178.0 (TID 44646) (nis-myfeed-exp1v3-w-1.c.inshorts-1374.internal executor 60): TaskKilled (Stage cancelled)
23/06/09 11:53:57 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 46.0 in stage 178.0 (TID 44665) (nis-myfeed-exp1v3-w-1.c.inshorts-1374.internal executor 70): TaskKilled (Stage cancelled)
23/06/09 11:53:57 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 6.0 in stage 178.0 (TID 44625) (nis-myfeed-exp1v3-w-1.c.inshorts-1374.internal executor 70): TaskKilled (Stage cancelled)
23/06/09 11:53:57 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 86.0 in stage 178.0 (TID 44705) (nis-myfeed-exp1v3-w-1.c.inshorts-1374.internal executor 70): TaskKilled (Stage cancelled)
23/06/09 11:53:57 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 126.0 in stage 178.0 (TID 44745) (nis-myfeed-exp1v3-w-1.c.inshorts-1374.internal executor 70): TaskKilled (Stage cancelled)
23/06

In [149]:
Log("Exectution finished successfully, in : %s minutes, %s seconds " % (int((time.time() - start_time) / 60), int((time.time() - start_time) % 60)), flag=True)

Exectution finished successfully, in : 211 minutes, 3 seconds 
